In [2]:
import ollama
ollama.list()

{'models': [{'name': 'llama3.1:latest',
   'model': 'llama3.1:latest',
   'modified_at': '2024-11-18T14:33:25.305661201+01:00',
   'size': 4661230766,
   'digest': '42182419e9508c30c4b1fe55015f06b65f4ca4b9e28a744be55008d21998a093',
   'details': {'parent_model': '',
    'format': 'gguf',
    'family': 'llama',
    'families': ['llama'],
    'parameter_size': '8.0B',
    'quantization_level': 'Q4_0'}}]}

In [15]:
from langchain_community.document_loaders import WebBaseLoader
import os

os.environ['USER_AGENT'] = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

In [14]:
loader_multiple_pages = WebBaseLoader(["https://www.home0001.com/how-it-works", "https://www.home0001.com/legal"])
docs = loader_multiple_pages.load()
print(docs[0].page_content[:128])

Learn how to own your home and live anywhere | Home0001HOME0001MenuHOME0001MenuHomes:0001: Lower East SideStudioStudio Max1 Bedr


In [6]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# set up the splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# split the docs
splits = text_splitter.split_documents(docs)
# create a vector database with the splits
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=OllamaEmbeddings(model="llama3.1"),
    # persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

print(len(splits))
print(len(splits[12].page_content))
print(splits[12].metadata)

99
990
{'source': 'https://www.home0001.com/legal', 'title': 'Legal Notices for 0001 homes live flexibly own your home', 'description': 'Own the perfect home.', 'language': 'en'}


In [7]:
# Retrieve and generate using the relevant snippets of the site.
# retriever = vectorstore.as_retriever()

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})
retrieved_docs = retriever.invoke("What is home0001?")

print(len(retrieved_docs))
print(retrieved_docs[0].page_content)

6
not always secure, please do not include credit card or other sensitive information in your emails to us.© 2024 HOME0001New york · los angeles · berlin · london · paris · mexico city0001 GDShow it workscontact usInstagramlegalCookies Settings© 2024 HOME0001NY DRE #10351211814CA DRE #01427385NY DRE #10351211814CA DRE #02236922WE USE COOKIES.Accept/Reject/Settings


In [8]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

/home/studio/anaconda3/envs/rag/lib/python3.10/site-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [9]:
from langchain_ollama import ChatOllama


llm = ChatOllama(
    model = "llama3.1",
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# print(format_docs(docs))

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in rag_chain.stream("What is Home0001?"):
    print(chunk, end="", flush=True)

print("/n")
print(rag_chain.invoke("can i rent an apartment?"))

Home0001 appears to be a home-sharing company that allows members to book homes in various locations. It seems to provide smart locks for secure access, community events, and a virtual concierge service. The company is registered in New York and has offices in several cities worldwide./n
No, you cannot rent an apartment from this context as it appears to be a real estate sales website with terms and conditions for purchasing homes, not renting apartments. The language used suggests buying and selling properties rather than rentals.
